In [28]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient
from pandas import json_normalize

Ya tenemos lo que queremos priorizar, lo cual es, en orden de mas prioritario a menos
- Big starup
- Vegan
- Starbucks
- Vet
- Bar

In [12]:
load_dotenv()

True

In [13]:
client_id = os.getenv('Client_Id')
client_secret = os.getenv('Client_Secret')

In [14]:
# me hago una funcion que me convierte las coordenadas en tipo point

In [15]:
def type_point(lista):
    return {"type": "Point", 
            "coordinates": lista
    }

In [16]:
url_query = 'https://api.foursquare.com/v2/venues/search'

In [115]:
#esto para los veganos en ny

In [32]:
parametros_v = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": '40.75673, -73.98996',
    "query": "Vegan",
    "limit": 200   
}

In [33]:
resp_vegan = requests.get(url_query, params=parametros_v).json()['response']['venues']

In [49]:
resp_vegan[0]['location'].keys()

dict_keys(['address', 'lat', 'lng', 'labeledLatLngs', 'distance', 'postalCode', 'cc', 'city', 'state', 'country', 'formattedAddress'])

In [113]:
#resp_vegan[0]

In [59]:
mapa_nombre = ['name']
mapa_latitud = ['location','lat']
mapa_longitud = ['location', 'lng']
mapa_ciudad = ['location','city']
mapa_distancia = ['location','distance']
print(mapa_ciudad)

['location', 'city']


In [57]:
def getFromDict(diccionario, mapa):
    from functools import reduce
    import operator
    return reduce(operator.getitem,mapa,diccionario)

In [85]:
getFromDict(resp_vegan[0], mapa_nombre)

'Omv! Vegan Burgers & More'

In [68]:
resp_vegan[0]["name"]

'Omv! Vegan Burgers & More'

In [109]:
vegan_json = []
for dic in resp_vegan:
    try:
        lista= {}
        lista['nombre']= getFromDict(dic,mapa_nombre)
        lista['latitud']= getFromDict(dic,mapa_latitud)
        lista['longitud']= getFromDict(dic,mapa_longitud)
        lista['ciudad']= getFromDict(dic,mapa_ciudad)
        lista['distancia'] = getFromDict(dic,mapa_distancia)
        vegan_json.append(lista)
    except:
        #print(dic)
    

SyntaxError: unexpected EOF while parsing (<ipython-input-109-0393abd75713>, line 13)

In [112]:
df_vegans_ny=pd.DataFrame(vegan_json)
df_vegans_ny.head()

,nombre,latitud,longitud,ciudad,distancia
0,Omv! Vegan Burgers & More,40.757819,-73.985537,New York,392
1,Marty's Vegan Fast Foods,40.750163,-73.988871,New York,736
2,MTHR Vegan,40.762373,-73.987867,New York,652
3,Happy Zoe Vegan Bakery,40.719246,-73.944252,New York,5681
4,Maoz Vegan,40.751706,-73.974375,New York,1428


In [125]:
def getAPI(url_query, coordenadas, query): #coordenadas en string, latitud y longitud (en ese orden)
    parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f'{coordenadas}',
    "query": f"{query}",
    "limit": 200   
}
    resp = requests.get(url_query, params=parametros).json()['response']['venues']
    
    return resp

In [144]:
def limpiezaresp(json):
    
    mapa_nombre = ['name']
    mapa_latitud = ['location','lat']
    mapa_longitud = ['location', 'lng']
    mapa_ciudad = ['location','city']
    mapa_distancia = ['location','distance']
    
    jason = []
    
    for dic in json:
        try:
            diccio= {}
            diccio['nombre']= getFromDict(dic,mapa_nombre)
            diccio['latitud']= getFromDict(dic,mapa_latitud)
            diccio['longitud']= getFromDict(dic,mapa_longitud)
            diccio['ciudad']= getFromDict(dic,mapa_ciudad)
            diccio['distancia'] = getFromDict(dic,mapa_distancia)
            jason.append(diccio)
        except:
            pass
    return pd.DataFrame(jason)

In [145]:
def todo(url_query, coordenadas, query):
    api = getAPI(url_query, coordenadas, query)
    return limpiezaresp(api)

In [148]:
df_vegans_ny = todo('https://api.foursquare.com/v2/venues/search','40.75673, -73.98996', 'Vegan')

In [150]:
df_vegans_ny.head()

,nombre,latitud,longitud,ciudad,distancia
0,Omv! Vegan Burgers & More,40.757819,-73.985537,New York,392
1,MTHR Vegan,40.762373,-73.987867,New York,652
2,Marty's Vegan Fast Foods,40.750163,-73.988871,New York,736
3,Happy Zoe Vegan Bakery,40.719246,-73.944252,New York,5681
4,Maoz Vegan,40.751706,-73.974375,New York,1428


In [151]:
lista = ['Vegan', 'Start Up', 'Veterinary', 'Bar', 'Starbucks']

In [152]:
for i in lista:
    df_ny = todo('https://api.foursquare.com/v2/venues/search','40.75673, -73.98996', i)

In [155]:
df_ny.shape

(50, 5)

In [147]:
# me creo dos listas, una con las coordenadas de cada place mio y otra con mis cosas que quiero buscar en la API 
# y hacer un zip sobre ello

In [116]:
# ahora para las startups en NY (buscar el valor de cada una en google)